## Откроем файл и изучим общую информацию

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})
# чтобы выполнить SQL-запрос, используем Pandas
query = '''SELECT * FROM books LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [2]:
query = '''SELECT * FROM authors LIMIT 5'''
pd.io.sql.read_sql(sql=text(query), con = con)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [3]:
query = '''SELECT * FROM ratings LIMIT 5'''
pd.io.sql.read_sql(sql=text(query), con = con)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [4]:
query = '''SELECT * FROM reviews LIMIT 5'''
pd.io.sql.read_sql(sql=text(query), con = con)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [5]:
query = '''SELECT * FROM publishers LIMIT 5'''
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


## Cколько книг вышло после 1 января 2000 года

In [6]:
query = '''SELECT DISTINCT COUNT(book_id) FROM books WHERE publication_date>'2000-01-01';'''
pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,819


Всего в данных 1000 книг, из ни, те которые были опубликованы после 1 января 2000 года - 819 книг.

## Посчитаем количество обзоров и среднюю оценку для каждой книги

In [7]:
query = '''
SELECT title,
       author,
       AVG(rating) AS avg_rating,
       COUNT(DISTINCT review_id) AS count_review
FROM books AS b 
INNER JOIN author AS a ON b.author_id=a.author_id
INNER JOIN ratings AS r ON b.book_id=r.book_id
INNER JOIN reviews AS rv ON b.book_id=rv.book_id
GROUP BY b.book_id, a.author
ORDER BY count_review DESC;
'''
pd.io.sql.read_sql(sql=text(query), con = con)

,title,author,avg_rating,count_review
0,Twilight (Twilight #1),Stephenie Meyer,3.662500,7
1,The Road,Cormac McCarthy,3.772727,6
2,The Book Thief,Markus Zusak/Cao Xuân Việt Khương,4.264151,6
3,The Glass Castle,Jeannette Walls,4.206897,6
4,Water for Elephants,Sara Gruen,3.977273,6
...,...,...,...,...
989,Naked Empire (Sword of Truth #8),Terry Goodkind,3.500000,1
990,Moo Baa La La La!,Sandra Boynton,3.000000,1
991,Merrick (The Vampire Chronicles #7),Anne Rice,4.000000,1
992,Babyville,Jane Green,3.500000,1


Самое большое количество обзоров у книги Stephenie Meyer Twilight (Twilight #1) - 7 обзоров.

## Определим издательство, которое выпустило наибольшее число книг толще 50 страниц(для исключения брошюр)

In [8]:
query = '''
SELECT publisher,
       COUNT(book_id) AS cnt_book
FROM publishers AS p 
LEFT JOIN books AS b ON p.publisher_id=b.publisher_id
WHERE num_pages>50
GROUP BY publisher
ORDER BY cnt_book DESC
LIMIT 1;
'''
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher,cnt_book
0,Penguin Books,42


Больше всего книг выпустило издательство Penguin Books - 42 штуки.

## Определим автора с самой высокой средней оценкой книг — учтем только книги с 50 и более оценками

In [9]:
query = '''
SELECT best_rating.author,
       AVG(best_rating.avg_rating)
FROM (SELECT AVG(rating) AS avg_rating,
       a.author
FROM books as b
INNER JOIN authors AS a ON b.author_id=a.author_id
INNER JOIN ratings AS r ON b.book_id=r.book_id
GROUP BY b.book_id, a.author
HAVING COUNT(rating_id)>=50
ORDER BY avg_rating DESC) AS best_rating
GROUP BY best_rating.author
LIMIT 1;
'''
pd.io.sql.read_sql(sql=text(query), con = con)

,author,avg
0,J.K. Rowling/Mary GrandPré,4.283844


Самая высокая средняя оценка книг с 50 или более оценками у J.K. Rowling/Mary GrandPré - 4,28.

## Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок

In [10]:
query = '''
SELECT AVG(cnt_rev.cnt_reviews)
FROM (SELECT username,
             COUNT(DISTINCT review_id) AS cnt_reviews
FROM reviews
WHERE username IN (SELECT username
                      FROM ratings
                      GROUP BY username
                      HAVING COUNT(rating_id)>48
                      ORDER BY username)
GROUP BY username) AS cnt_rev;
'''
pd.io.sql.read_sql(sql=text(query), con = con)

,avg
0,24.0


Среднее количество обзоров от пользователей, которые поставили больше 48 оценок составляет 24 обзора.

## Вывод
Проделав небольшой анализ баз данных приложения для чтения книг можно сказать:

* В приложении находится база из 1000 книг, включая информацию о самих книгах, авторах, издательствах. Есть информация о рейтингах книг, отзывах и тех, кто их составил, и даже текст самого обзора. Удобно, что помимо самих книг, есть много интересной информации о них и пользователям думают будет интересно с ней ознакомиться, так же это очень помогает в выборе;
* Так же в ходе исследования баз данных определили издательство которое выпустило больше всего книг, исключая брошюры - это издательство Penguin Books;
* Узнали кто из авторов имеет самую высокой среднюю оценку книг - это J.K. Rowling/Mary GrandPré с рейтингом 4,28
* Так же узнали, что среднее количество обзоров у которых больше 48 оценок - 24. 